# Stock Market And Forecasting Stacked LSTM Prediction Using 💹

In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Loading the dataset and preprocess it
data_frame = pd.read_csv('NSE-TATAGLOBAL.csv', parse_dates=['Date'], index_col='Date')
data_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = data_scaler.fit_transform(data_frame)

In [4]:
# Creating a function to create a stacked LSTM model
def create_model(units, n_features):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(None, n_features)))
    model.add(LSTM(units))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [5]:
# Spliting the dataset into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size, :]
test_data = scaled_data[train_size:, :]

In [6]:
# Creating sequences and labels for the training data
n_steps = 60
x_train, y_train = [], []
for i in range(n_steps, len(train_data)):
    x_train.append(train_data[i - n_steps:i, :])
    y_train.append(train_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [7]:
# Creating sequences and labels for the testing data
x_test, y_test = [], []
for i in range(n_steps, len(test_data)):
    x_test.append(test_data[i - n_steps:i, :])
    y_test.append(test_data[i, 0])
x_test, y_test = np.array(x_test), np.array(y_test)

In [8]:
model = create_model(units=50, n_features=scaled_data.shape[1])
model.fit(x_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
49/49 [==============================] - 4s 23ms/step - loss: 0.0088
Epoch 2/50
49/49 [==============================] - 1s 23ms/step - loss: 9.0796e-04
Epoch 3/50
49/49 [==============================] - 1s 23ms/step - loss: 8.5684e-04
Epoch 4/50
49/49 [==============================] - 1s 23ms/step - loss: 8.1638e-04
Epoch 5/50
49/49 [==============================] - 1s 25ms/step - loss: 7.7643e-04
Epoch 6/50
49/49 [==============================] - 1s 23ms/step - loss: 7.3040e-04
Epoch 7/50
49/49 [==============================] - 1s 23ms/step - loss: 7.2458e-04
Epoch 8/50
49/49 [==============================] - 1s 23ms/step - loss: 6.2525e-04
Epoch 9/50
49/49 [==============================] - 1s 23ms/step - loss: 6.2597e-04
Epoch 10/50
49/49 [==============================] - 1s 23ms/step - loss: 6.1855e-04
Epoch 11/50
49/49 [==============================] - 1s 23ms/step - loss: 5.6156e-04
Epoch 12/50
49/49 [==============================] - 1s 23ms/step - loss: 5.50

In [9]:
# Evaluating the performance of the model on the testing data
mse = model.evaluate(x_test, y_test)
print("Mean Squared Error: {:.2f}".format(mse))

11/11 [==============================] - 1s 15ms/step - loss: 1.4012e-04
Mean Squared Error: 0.00


In [10]:
# Making predictions on the testing data using the trained model
y_pred = model.predict(x_test)

11/11 [==============================] - 1s 13ms/step


In [11]:
# have to Inverse scale the predictions and actual values
y_pred = data_scaler.inverse_transform(np.concatenate((y_pred, x_test[:, -1, 1:]), axis=1))[:, 0]
y_test = data_scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), x_test[:, -1, 1:]), axis=1))[:, 0]

In [12]:
# And at last Printing the predicted and actual values
predictions = pd.DataFrame({'Date': data_frame.index[train_size + n_steps:], 'Predicted Close': y_pred, 'Actual Close': y_test})
print(predictions)

          Date  Predicted Close  Actual Close
0   2011-12-09        90.091955         88.75
1   2011-12-08        90.515857         90.85
2   2011-12-07        91.027194         91.00
3   2011-12-05        91.595391         91.70
4   2011-12-02        92.116585         88.60
..         ...              ...           ...
342 2010-07-27       116.625395        117.60
343 2010-07-26       117.316702        120.10
344 2010-07-23       118.268563        121.80
345 2010-07-22       119.903231        120.30
346 2010-07-21       120.609233        122.10

[347 rows x 3 columns]
